In [1]:
import pandas as pd

df= pd.read_csv("/kaggle/input/fbref-data/FBREF Data.csv")

Next, I will be separating the table into two types of columns: positive impact on team and negative impact on team.

In [2]:
df.rename(columns={"03-Jan":"1/3"}, inplace= True)

In [3]:
df.drop(df[df.columns[[0]]], axis=1, inplace= True)

In [4]:
#removing entries with 0 matches played as they would be irrelevent to the final outcome.

df.drop(df[df['90s'] == 0].index, inplace=True)
df.drop(df[df['Pos'] == 'GK'].index, inplace=True)

df.reset_index(drop=True, inplace=True)

In [5]:
#creating table with all necessary columns
features= df[['90s','Player', 'Pos', 'Squad', 'Nation', 'Comp', 'Season',
 'Standard Gls',
 'Standard SoT',
 'Standard SoT%',
 'Standard PK',
 'Standard G/Sh',
 'Expected npxG',
 'Expected np:G-xG',
 'Total TotDist',
 'Total PrgDist',
 'Total Cmp',
 'Total Cmp%',
 'Short Cmp',
 'Short Att',
 'Short Cmp%',
 'Medium Cmp',
 'Medium Att',
 'Medium Cmp%',
 'Long Cmp',
 'Long Att',
 'Long Cmp%',
 'Ast',
 'Expected xA',
 'KP',
 '1/3',
 'PPA',
 'CrsPA',
 'PrgP',
 'SCA SCA',
 'SCA SCA90',
 'GCA GCA',
 'GCA GCA90',
 'Tackles TklW',
 'Challenges Tkl',
 'Challenges Att',
 'Challenges Lost',
 'Blocks Blocks',
 'Blocks Sh',
 'Blocks Pass',
 'Int',
 'Clr',
 'Err',
 'Touches Def 3rd',
 'Touches Mid 3rd',
 'Touches Att 3rd',
 'Take-Ons Succ',
 'Take-Ons Succ%',
 'Take-Ons Tkld',
 'Take-Ons Tkld%',
 'Carries TotDist',
 'Carries PrgDist',
 'Carries PrgC',
 'Carries 1/3',
 'Carries CPA',
 'Carries Mis',
 'Carries Dis',
 'Receiving Rec',
 'Receiving PrgR',
 'Performance CrdY',
 'Performance CrdR',
 'Performance 2CrdY',
 'Performance Fls',
 'Performance Fld',
 'Performance Off',
 'Performance PKwon',
 'Performance PKcon',
 'Performance OG',
 'Performance Recov',
 'Aerial Duels Won',
 'Aerial Duels Lost']]



In [6]:
#creating separate tables with positive and negative impact columns only
positive= features.drop(columns= ['Player','Pos', 'Squad','Challenges Lost', 'Err','Nation', 'Comp',  'Take-Ons Tkld', 'Take-Ons Tkld%',
 'Carries Mis', 'Carries Dis',
 'Performance CrdY', 'Performance CrdR',
 'Performance 2CrdY', 'Performance Fls',
 'Performance Off', 'Performance PKcon',
 'Performance OG', 'Aerial Duels Lost', 'Season'])

negative= features[['Challenges Lost',
 'Err',
 'Carries Mis',
 'Carries Dis',
 'Take-Ons Tkld',
 'Take-Ons Tkld%',
 'Performance CrdY',
 'Performance CrdR',
 'Performance 2CrdY',
 'Performance Fls',
 'Performance Off',
 'Performance PKcon',
 'Performance OG',
 'Aerial Duels Lost']]

# Standardizing Metrics

## Positive Metrics

In [7]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the features
scaled_features = scaler.fit_transform(positive)

# Convert the scaled features back into a DataFrame
scaled_positive = pd.DataFrame(scaled_features, columns=positive.columns)


## Negative metrics

In [8]:
# Fit and transform the features
scaled_features = scaler.fit_transform(negative)

# Convert the scaled features back into a DataFrame
scaled_negative = pd.DataFrame(scaled_features, columns=negative.columns)

## Adding up all the scores

My methodology:

Sohom Score   =   (Positive Metrics) - (Negative Metrics)

In [9]:
seasonal_score= pd.DataFrame()

scaled_positive['Positive Score'] = scaled_positive.sum(axis=1)
seasonal_score['Positive Score']= scaled_positive[['Positive Score']]

scaled_negative['Negative Score'] = scaled_negative.sum(axis=1)
seasonal_score['Negative Score']= scaled_negative[['Negative Score']]

In [10]:
seasonal_score['Player']= df['Player'] 
seasonal_score['Pos']= df['Pos'] 
seasonal_score['Squad']= df['Squad'] 
seasonal_score['Season']= df['Season'] 
seasonal_score['Comp']= df['Comp'] 


seasonal_score["Sohom's Seasonal Score"]= seasonal_score['Positive Score']-seasonal_score['Negative Score']

In [11]:
seasonal_score.sort_values(by= "Sohom's Seasonal Score", ascending= False).head(10)

,Positive Score,Negative Score,Player,Pos,Squad,Season,Comp,Sohom's Seasonal Score
9378,176.822470,14.817793,Lionel Messi,"FW,MF",Barcelona,2020,La Liga,162.004677
16794,172.621836,12.668453,Lionel Messi,FW,Barcelona,2017,La Liga,159.953383
15835,165.958302,10.612456,Kevin De Bruyne,MF,Manchester City,2017,Premier League,155.345846
11881,167.708301,12.705179,Lionel Messi,FW,Barcelona,2019,La Liga,155.003123
14333,162.689780,14.245045,Lionel Messi,FW,Barcelona,2018,La Liga,148.444735
13790,155.311859,13.377208,Eden Hazard,FW,Chelsea,2018,Premier League,141.934651
10947,141.589335,8.782046,Kevin De Bruyne,MF,Manchester City,2019,Premier League,132.807289
10410,137.747281,8.619124,Luis Alberto,MF,Lazio,2019,Serie A,129.128157
13998,134.892651,7.592742,Joshua Kimmich,"DF,MF",Bayern Munich,2018,Bundesliga,127.299909
5567,137.581926,11.553165,João Cancelo,DF,Manchester City,2021,Premier League,126.028760


In [12]:
seasonal_score.to_csv("Standardized Data.csv")